In [1]:
DEVICE = 'cuda:2'
BATCH_SIZE = 64

In [2]:
import os
import sys
import torch
sys.path.append("../../")
torch.cuda.set_device(int(DEVICE[-1]))

In [3]:
# import libraries
from experiments.lapq.datasets import DataManager
from trailmet.models import resnet
from trailmet.algorithms.quantize.lapq import LAPQ

In [4]:
data_object = DataManager()
trainloader, valloader, testloader = data_object.prepare_data()
dataloaders = {'train': trainloader, 'calib': valloader, "val": testloader}

... Preparing data ...
File already downloaded
File already downloaded
File already downloaded
using fixed split
90000 10000


In [5]:
# load model
cnn = resnet.get_resnet_model('resnet50', 200, 64, pretrained=False)
checkpoint = torch.load("./resnet50_tinyimagenet_pretrained.pth", map_location=DEVICE)
cnn.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [6]:
# test model
from trailmet.algorithms.algorithms import BaseAlgorithm
BaseAlgorithm().test(model=cnn, dataloader=dataloaders['val'], device=torch.device(DEVICE))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:12<00:00, 13.05it/s, acc1=59.6, acc5=81.5]


(59.613853503184714, 81.47890127388536)

In [7]:
# quantize model
kwargs = {
    'W_BITS':8, 
    'A_BITS':8, 
    'ACT_QUANT':True,
    'CALIB_BATCHES':1024//BATCH_SIZE, 
    'MAX_ITER':100,
    'MAX_FEV':100,
    'VERBOSE':True,
    'PRINT_FREQ':20,
    'GPU_ID':int(DEVICE[-1]),
    'SEED':42
    }
qnn = LAPQ(cnn, dataloaders, **kwargs)
qnn.compress_model()

==> Using seed: 42 and device: cuda:2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [01:37<00:00,  1.61it/s, acc1=58.7, acc5=81]


==> Quantization (W8A8) accuracy before LAPQ: 58.7480 | 80.9713


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [03:11<00:00, 19.11s/it, loss=0.714, p_val=4]


==> using p intr : 3.42


...

==> Quantization (W8A8) accuracy before Optimization: 58.4893 | 80.8221
==> Loss after LpNorm Quantization: 1.7571
==> Starting Powell Optimization


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [15:27<00:00,  9.28s/it, curr_loss=0.802, min_loss=0.801]


==> Layer-wise Scales :
 [0.76732412 0.1708154  0.73032549 3.5942775  1.38828419 0.42208314
 0.43678823 1.63470995 0.8616091  0.30070755 0.40909269 1.18723071
 0.54597455 0.45366174 0.44391575 0.87166148 0.85467261 0.69831443
 0.3288452  0.36065897 1.70181191 0.52933598 0.42445689 0.47822899
 0.64057261 0.49391904 0.31407401 0.49710608 0.87855458 0.62337083
 0.4534944  0.17147495 0.72989857 1.29332173 0.61206615 0.19430335
 0.40519947 1.25222135 0.67409742 0.37798271 0.32127845 1.05952668
 0.52090627 0.29425791 0.1359327  0.89971143 0.55755585 0.14317717
 0.17550938 0.8929401  0.68530428 0.39004546 0.26673165 1.07781041
 0.78103995 0.34187454 0.17926063 1.01454782 1.14515138 0.45034611
 0.1075656  0.18343365 1.73643827 0.94523704 0.14316259 0.2444692 ]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [01:28<00:00,  1.78it/s, acc1=58.6, acc5=81]


==> Full quantization (W8A8) accuracy: (58.55891719745223, 80.9812898089172)


In [8]:
# test quantized model
from trailmet.algorithms.algorithms import BaseAlgorithm
BaseAlgorithm().test(model=qnn.model, dataloader=dataloaders['val'], device=torch.device(DEVICE))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [01:26<00:00,  1.81it/s, acc1=58.6, acc5=81]


(58.55891719745223, 80.9812898089172)